In [2]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time

import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat,loadmat



#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device1 = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device2 = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

print("Device1 ",device1)
print("device2 ",device2)



Device1  cuda:1
device2  cuda:2


In [3]:
# def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
#     y = np.exp(-4.0*x) + np.exp(3.0*x)
#     return y
import websocket
import threading
# import numpy as np
import base64
import uuid
import json
# import time

run_uuid = "0e8254d2-dda0-402c-87d2-c4bcad726a97" #Random fixed https://www.uuidgenerator.net/version4
client_uuid = uuid_v4 = uuid.uuid4()

def get_random_loss():
    return np.float32(np.random.rand(6,1))

def initialize_message(client_uuid, run_uuid):
    return '{"messageType":"initialize","clientUUID":"' + str(client_uuid) + '","runUUID":"' + str(run_uuid) + '"}'


def update_message(client_uuid, run_uuid, data):
    base64_data = base64.b64encode(data.tobytes())
    return '{"messageType":"update","clientUUID":"' + str(client_uuid) + '","runUUID":"' + str(run_uuid) + '", "data":"' + str(base64_data, "ascii") + '"}'

class WebSocketClient:
    def __init__(self, url):
        self.url = url
        self.response = None
        self.ws = websocket.WebSocketApp(
            url,
            on_message=self.on_message,
            on_open=self.on_open,
            on_error=self.on_error,
            on_close=self.on_close
        )
        self.ws_thread = threading.Thread(target=self.ws.run_forever)
        self.connected = threading.Event()

    def on_message(self, ws, message):
        print("|", message, "|")
        json_data = json.loads(message)
        self.response = np.frombuffer(base64.b64decode(json_data["data"]), dtype=np.float32)
        
    def on_open(self, ws):
        self.connected.set()

    def on_error(self, ws, error):
        print(f"WebSocket error: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        self.connected.clear()
        print("WebSocket closed")

    def send_message(self, message):
        self.response = None
        self.ws.send(message)

    def start(self):
        self.ws_thread.start()
        self.connected.wait()  # Wait until the connection is open

    def stop(self):
        self.ws.close()
        self.ws_thread.join()

In [4]:
level = "high"
label = "1D_SODE_Stan" + level

#MATLAB Van Der Pol Example https://www.mathworks.com/help/matlab/ref/ode89.html
mu = 1
fo_val = 0.0

loss_thresh = 0.005

x = np.linspace(0,10,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y1 = np.array([2]).reshape(-1,1)
x1_bc1_train = torch.from_numpy(bc1_x).float()#.to(device1)
y1_bc1_train = torch.from_numpy(bc1_y1).float()#.to(device1)
    

bc1_x = x[0].reshape(-1,1)
bc1_y2 = np.array([fo_val]).reshape(-1,1)
x2_bc1_train = torch.from_numpy(bc1_x).float()#.to(device2)
y2_bc1_train = torch.from_numpy(bc1_y2).float()#.to(device2)    


x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float()#.to(device)
# y_true = true_1D_1(x_test)
# y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [5]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [6]:
def train_step(model_PINN,optimizer,x1_bc1,y1_bc1,x2_bc1,y2_bc1,x_coll1,x_coll2,f_hat1,f_hat2):
    
    
    optimizer.zero_grad()
    loss1 = model_PINN.loss_y1(x1_bc1,y1_bc1,x_coll1,f_hat1)
    
    ws_client.send_message(update_message(client_uuid, run_uuid, loss1))
    while ws_client.response is None:
        pass
    
    aggregated_loss = ws_client.response.copy().reshape(6,1)
    # print("recieved", aggregated_loss)
    ws_client.response = None


    # loss2 = model_PINN.loss_y2(x2_bc1,y2_bc1,x_coll2,f_hat2)
    # loss1.backward()
    aggregated_loss.backward()
    # loss2.backward()
    optimizer.step()

    # optimizer1.step(closure1)
    # optimizer2.step(closure2)


In [7]:
# def data_update(loss_np):
#     train_loss.append(loss_np)
#     beta_val.append(PINN.beta.cpu().detach().numpy())
    
#     test_mse, test_re = PINN.test_loss()
#     test_mse_loss.append(test_mse)
#     test_re_loss.append(test_re)

In [8]:
def train_model(model_PINN,optimizer,max_iter,rep,N_f):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0
    
    x_coll1 = torch.from_numpy(colloc_pts(N_f,0)).float()#.to(device1)
    x_coll2 = torch.from_numpy(colloc_pts(N_f,1)).float()#.to(device2)
    
    f_hat1 = torch.zeros(x_coll1.shape[0],1).to(device1)
    f_hat2= torch.zeros(x_coll2.shape[0],1).to(device2)

    loss_np1 = model_PINN.loss_y1(x1_bc1_train,y1_bc1_train,x_coll1,f_hat1).cpu().detach().numpy()
    loss_np2 = model_PINN.loss_y2(x2_bc1_train,y2_bc1_train,x_coll2,f_hat2).cpu().detach().numpy()
    
    # data_update(loss_np1)
    for i in range(max_iter):
        # x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        # f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(model_PINN,optimizer,x1_bc1_train,y1_bc1_train,x2_bc1_train,y2_bc1_train,x_coll1,x_coll2,f_hat1,f_hat2)
        
        if(i%100==0):
            loss_np1 = model_PINN.loss_y1(x1_bc1_train,y1_bc1_train,x_coll1,f_hat1).cpu().detach().numpy()
            loss_np2 = model_PINN.loss_y2(x2_bc1_train,y2_bc1_train,x_coll2,f_hat2).cpu().detach().numpy()
                
            # data_update(loss_np1)
            # print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
            print(i,"Loss1",loss_np1,"Loss2",loss_np2)
        
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
from models import coupled_PINN
max_reps = 1
max_iter = 2000 #5000 for Adam

N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):
    
    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    beta_val = []
    
    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 
    
    
    layers1 = np.array([1,50,50,50,1])
    layers2 = np.array([1,50,50,50,50,1])
    # layers = np.array([1,50,50,50,50,1])
    model_PINN = coupled_PINN(layers1,layers2,device1,device2)

    'Neural Network Summary'

    optimizer = torch.optim.Adam(model_PINN.parameters(),lr=0.01, betas=(0.9, 0.999))

    # Usage
    ws_client = WebSocketClient("ws://128.173.95.177:8087")

    # Start WebSocket communication in a separate thread
    ws_client.start()

    # Send a message and wait for the response
    ws_client.send_message(initialize_message(client_uuid, run_uuid))


    train_model(model_PINN,optimizer,max_iter,reps,N_f)

    
    # torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    # train_loss_full.append(train_loss)
    # test_mse_full.append(test_mse_loss)
    # test_re_full.append(test_re_loss)
    # beta_full.append(beta_val)    
    
    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "beta": beta_full, "label": label, "Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1-2): 2 x Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=1, bias=True)
  )
)
Sequentialmodel(
  (activation): Tanh()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1-3): 3 x Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)
WebSocket error: [Errno 111] Connection refused
WebSocket closed


In [ ]:
u_pred = model_PINN.test(x_test_tensor)
plt.plot(x,u_pred,'r')
# plt.plot(y_true,'b')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cpu!

In [ ]:
# data_mat = loadmat('Vanderpol_ODEsolver.mat')
# y = data_mat['y'][:,0]
# t = data_mat['t']

# fig,ax = plt.subplots()
# ax.plot(t,y,'b',linewidth = 2,label = 'Numerical Solver')
# ax.plot(x,u_pred,'r-.',linewidth = 2,label = 'Coupled PINN')
# ax.set_xlabel('Time(s)')
# ax.set_ylabel('Value')
# ax.set_title('van der Pol Oscillator')
# ax.legend(loc = 3)
# plt.savefig('Coupled_PINN_VanderPol.svg',format = 'svg')
# plt.savefig('Coupled_PINN_VanderPol.png',format = 'png')